In [1]:
import sys
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install textblob
import config as config

import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
  
class TwitterClient(object):
    def __init__(self):
        consumer_key = config.consAPIkey
        consumer_secret = config.consAPIsecretkey
        access_token = config.accesstoken
        access_token_secret = config.accesstokensecret
        
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed")
            
    def clean_tweet(self, tweet): 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
          
        
    def get_tweet_sentiment(self, tweet): 
        
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
    
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet 
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))
     
        # creating object of TwitterClient Class 
api = TwitterClient() 
        # calling function to get tweets 
tweets = api.get_tweets(query = 'Dhoni', count = 200) 
  
        # picking positive tweets from tweets 
ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
        # percentage of positive tweets 
print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
        # picking negative tweets from tweets 
ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']

neutraltweets = [tweet for tweet in tweets if not tweet['sentiment'] == 'positive' and not tweet['sentiment']== 'negative' ]
    # percentage of negative tweets 
print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
print("Neutral tweets percentage: {} %".format(100*len(neutraltweets)/len(tweets)))
  
        # printing first 5 positive tweets 
print("\n\nPositive tweets:") 
for tweet in ptweets[0:10]: 
    print(tweet['text']) 
  
print("\n\nNegative tweets:") 
for tweet in ntweets[0:10]: 
            print(tweet['text'])

ModuleNotFoundError: No module named 'config'